In [1]:
!pip install chromadb
!pip install sentence_transformers
!pip install ultralytics
!pip install langchain_community
!pip install langchain
!pip install accelerate
!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 29

In [2]:
import torch
import torch.nn.functional as F
import re
import os
from chromadb.config import Settings
from chromadb.api.models.Collection import Collection
import pandas as pd
from tqdm import tqdm
import chromadb
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain_community.document_loaders.csv_loader import CSVLoader
from transformers import AutoModel, AutoTokenizer, Conversation, pipeline
from langchain.document_loaders import DataFrameLoader
from annoy import AnnoyIndex
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import pickle as pkl
import torch

In [3]:
MODEL_EMB_NAME = "intfloat/multilingual-e5-large-instruct"
DEVICE = "cuda"
MODEL_CHAT_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
SYSTEM_PROMPT = """
INSTRUCT:
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don’t know the answer to a question, please don’t share false information.

If you receive a question that is harmful, unethical, or inappropriate, end the dialogue immediately and do not provide a response.

If you make a mistake, apologize and correct your answer.

Generate a response based solely on the provided document.

Answer the following question language based only on the CONTEXT provided.

Отвечай только на русском языке.
"""

In [5]:
data = pd.read_csv('QAdata')
question = list(data.QUESTION)
answer = list(data.ANSWER)
search_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
with open('model.pkl', 'rb') as f:
    embeddings_search = pkl.load(f)
annoy_index = AnnoyIndex(len(embeddings_search[1]), 'angular')
annoy_index.load('Annoy_index')

True

In [6]:
client = chromadb.PersistentClient(path="/content/chroma")
collection = client.get_or_create_collection("database", metadata={"hnsw:space": "cosine"})

In [7]:
def load_models(model: str, device: str = "cpu", torch_dtype: str = "auto") -> tuple:
    tokenizer = AutoTokenizer.from_pretrained(model, device_map=device, torch_dtype=torch_dtype)
    model = AutoModel.from_pretrained(model, device_map=device, torch_dtype=torch_dtype)
    return tokenizer, model

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


def txt2embeddings(text: str, tokenizer, model, device: str = "сpu") -> torch.Tensor:
    # Кодируем входной текст с помощью токенизатора
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

    # Перемещаем закодированный ввод на указанное устройство
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}

    # Получаем выход модели для закодированного ввода
    with torch.no_grad():
      model_output = model(**encoded_input)

    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    embeddings = F.normalize(embeddings, p=2, dim=1)
    return embeddings.tolist()

In [8]:
emb_tokenizer, emb_model = load_models(MODEL_EMB_NAME, device=DEVICE)

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [9]:
df223 = pd.read_csv("/content/223.csv")
df44 = pd.read_csv("/content/44.csv")
df = pd.concat([df223, df44])
loader = DataFrameLoader(df, page_content_column='article_text')
documents = loader.load()

In [10]:
text_data = [item.page_content for item in documents]

In [11]:
def fill_documents(documents):
    df = pd.DataFrame()
    df["rule_name"] = [doc.metadata['rule_name'] for doc in documents]
    df["article_name"] = [doc.metadata['article_name'] for doc in documents]
    df["article_text"] = [doc.page_content for doc in documents]
    return df
result_data = fill_documents(documents)

In [12]:
def collectionUpsert(df, tokenizer, model, device: str = "сuda"):
  article_texts = df["article_text"].tolist()
  article_names = df["article_name"].tolist()
  rule_names = df["rule_name"].tolist()
  embeddings = []
  for text in article_texts:
      res = txt2embeddings(text, tokenizer, model, device)
      embeddings.append(res[0])

  ids = [str(i) for i in range(1, len(df['article_name'])+1)]
  metadatas = [
        {
            "source": "dataset",
            "article_name": article_name,
            "rule_name": rule_name,
        }
        for article_name, rule_name in zip(article_names, rule_names)
    ]
  collection.upsert(
      ids=ids,
      embeddings=embeddings,
      metadatas=metadatas,
      documents=article_texts
  )

  return collection



In [13]:
collection_result = collectionUpsert(result_data, emb_tokenizer, emb_model, DEVICE)

In [14]:
def encodeQuestion(question, tokenizer, model, device, collection):
  embedding = txt2embeddings(question, tokenizer, model, device)
  result = collection.query(query_embeddings=embedding, n_results=2)
  result_documents = []

  for distance, metadata, document in zip(
          result["distances"][0], result["metadatas"][0], result["documents"][0]
  ):
      if distance < 1:
          result_documents.append({"article_name": metadata["article_name"], "rule_name": metadata["rule_name"], "answer": document, "metric":(1 - float(distance))*100})
  return result_documents

In [15]:
def load_chatbot(model: str, device: str = "cuda", torch_dtype: str = "auto"):
    # Загружаем чатбот с помощью pipeline из библиотеки transformers
    chatbot = pipeline(
        model=model,
        trust_remote_code=True,
        torch_dtype=torch_dtype,
        device_map=device,
        task="conversational",
    )
    return chatbot


def append_documents_to_conversation(conversation, texts):
    text = "\n".join(texts)
    document_template = f"""
    CONTEXT:
    {text}
    Отвечай только на русском языке.
    ВОПРОС:
    """
    conversation.add_message({"role": "user", "content": document_template})

    return conversation


def generate_answer(
    chatbot,
    conversation: Conversation,
    max_new_tokens: int = 128,
    temperature=0.9,
    top_k: int = 50,
    top_p: float = 0.95,
    repetition_penalty: float = 2.0,
    do_sample: bool = True,
    num_beams: int = 2,
    early_stopping: bool = True,
) -> str:
    # Генерируем ответ от чатбота
    conversation = chatbot(
        conversation,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        do_sample=do_sample,
        num_beams=num_beams,
        early_stopping=early_stopping,
    )

    return conversation

In [16]:
chatbot = load_chatbot(MODEL_CHAT_NAME, device="cuda")

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [17]:
def get_answer(question: str,  tokenizer, model, device, collection, chatbot):
  conversation = Conversation()
  conversation.add_message({"role": "system", "content": SYSTEM_PROMPT})
  answers = encodeQuestion(question, tokenizer, model, device, collection)
  answer =  [i['answer'] for i in answers]
  conversation = append_documents_to_conversation(conversation, answer)
  conversation.add_message({"role": "user", "content": question})
  output = generate_answer(chatbot, conversation, temperature=0.9)
  return output[-1]["content"]

In [53]:
data = pd.read_csv('QAdata')
questions = list(data.QUESTION)
answer = list(data.ANSWER)

In [49]:
question = "Какие штрафы предусмотрены за неисполнение контракта в 44-ФЗ?"

In [57]:
def question_response(embeddings, question, emb_tokenizer, emb_model, device, collection, chatbot):
    top_k_hits = 5
    question_embedding = search_model.encode(question)
    corpus_ids, scores = annoy_index.get_nns_by_vector(question_embedding, top_k_hits, include_distances=True)
    hits = []
    for id, score in zip(corpus_ids, scores):
        hits.append({'corpus_id': id, 'score': 1-((score**2) / 2)})
    hits_above_threshold = [hit for hit in hits if hit['score'] > 0.8]

    result_answer = None

    if hits_above_threshold:
        question_embedding_tensor = torch.tensor(question_embedding)
        embeddings = torch.tensor(embeddings)
        correct_hits = util.semantic_search(question_embedding, embeddings, top_k=top_k_hits)[0]
        correct_hits_ids = list([hit['corpus_id'] for hit in correct_hits])
        with open('user_questions.txt', 'a') as f:
            f.write(f"Вопрос пользователя: {question}\n")
        result_answer = answer[correct_hits_ids[0]]

    if result_answer is None:
        result_answer = get_answer(question, emb_tokenizer, emb_model, device, collection, chatbot)

    return result_answer

In [58]:
print(question_response(embeddings_search, question, emb_tokenizer, emb_model, DEVICE, collection, chatbot))

Перечень штрафов приведен в Постановлении Правительства РФ от 30.08.2017 N 1042 (ред. от 02.08.2019), а пеня начисляется по нормам Гражднаского Кодекса РФ как одна трехсотая действующей на дату уплаты пени ставки рефинансирования Центрального банка Российской Федерации от цены контракта, уменьшенной на сумму, пропорциональную объему обязательств, предусмотренных контрактом и фактически исполненных поставщиком (подрядчиком, исполнителем)
